In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
#
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed

import tools
import importlib
importlib.reload(tools)
#
from datetime import datetime
from dateutil.relativedelta import relativedelta
from sklearn.metrics import r2_score, mean_squared_error
#
import warnings
warnings.filterwarnings("ignore")

/home/ec2-user/anaconda3/envs/slacktrading/lib/python3.7/site-packages/mpl_finance.py:22: DeprecationWarning: 



    Please use `mplfinance` instead (no hyphen, no underscore).

    To install: `pip install --upgrade mplfinance` 

   For more information, see: https://pypi.org/project/mplfinance/


  category=DeprecationWarning)


### Download data from AWS S3

In [2]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.download_data_from_s3(s3_bucket, local_path)

### CPU Count

In [3]:
cpu_count()

16

### Begin

In [4]:
# dataset = pd.read_csv('../data/all_stock_close.csv')
dataset = pd.read_csv('../data/us_hk/us_hk_clean_10yr_stock_close.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)
dataset.head(3)

,0001.HK,0002.HK,0003.HK,0004.HK,0005.HK,0006.HK,0007.HK,0008.HK,0010.HK,0011.HK,...,VFC,VRNT,WDC,WFC,WMT,^DJI,^GSPC,^IXIC,^TNX,^VIX
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-28,56.334431,41.966953,5.800245,7.799201,49.283875,26.950289,0.71,1.890068,40.080761,82.240196,...,16.199192,31.690001,26.569702,23.259796,41.967972,11575.540039,1258.510010,2662.879883,3.481,17.520000
2010-12-29,57.905922,42.365368,5.895123,8.097270,49.750565,27.142986,0.72,1.935749,40.827366,83.661453,...,16.249552,31.540001,26.420523,23.125603,42.233482,11585.379883,1259.780029,2666.929932,3.341,17.280001
2010-12-30,57.620216,42.199368,5.888801,8.229015,49.812798,27.170521,0.76,1.952879,41.063133,83.726082,...,16.201061,31.500000,26.420523,22.976505,42.225681,11569.709961,1257.880005,2662.979980,3.369,17.520000


In [5]:
train_start_txt = '2011-01-01'
train_end_txt = '2011-12-31'
dataset = dataset[train_start_txt:train_end_txt]

In [6]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))
#
stocks_prev = prevday_returns.columns.to_list()
stocks_today_draft = today_returns.columns.to_list()
stocks_today = [s for s in stocks_today_draft if '.HK' in s]
#
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col
#
comb_df = pd.concat([prevday_returns, today_returns[stocks_today]], axis=1)
comb_df.shape

(245, 1525)

### Fina the NaN in comb_df

In [7]:
comb_df.dropna(axis=0, how='any', inplace=True)

In [8]:
comb_df.tail(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-12-28,0.012451,0.002274,0.007808,0.011281,0.014316,0.003512,-0.029853,0.054347,0.042260,0.007547,...,-0.048374,0.005141,-0.002238,-0.017483,-0.020762,-0.057175,0.000000,-0.013129,-0.007435,0.000000
2011-12-29,0.004829,-0.007599,-0.001112,-0.025573,-0.010084,0.006987,-0.015268,-0.003839,-0.005764,-0.004845,...,-0.014685,0.005115,-0.009009,-0.008854,0.000000,-0.060644,-0.020495,-0.013304,-0.030305,0.000000
2011-12-30,-0.009142,-0.000763,-0.006697,-0.004321,-0.013606,-0.008741,0.000000,0.041437,0.009206,-0.003243,...,-0.002963,0.000000,0.011247,-0.002545,0.000000,0.459543,-0.017911,-0.018019,0.015268,0.013247


In [9]:
comb_df.head(3)

,0001.HK.prev,0002.HK.prev,0003.HK.prev,0004.HK.prev,0005.HK.prev,0006.HK.prev,0007.HK.prev,0008.HK.prev,0010.HK.prev,0011.HK.prev,...,3900.HK,3918.HK,3933.HK,3968.HK,3988.HK,3989.HK,3993.HK,3998.HK,3999.HK,6823.HK
Date,,,,,,,,,,,,,,,,,,,,,
2011-01-05,0.012648,-0.005488,0.002165,0.001615,0.003118,0.001009,-0.027029,0.008607,0.000955,-0.001539,...,0.006369,0.000000,0.002567,0.002462,-0.007203,-0.028982,0.010612,0.040191,0.000000,-0.008772
2011-01-06,0.010938,0.004706,-0.001083,-0.011347,0.023994,0.001009,-0.013793,-0.014388,0.004764,0.017564,...,0.015748,0.056834,-0.033907,0.000000,0.002407,0.000000,-0.017302,-0.021440,-0.006473,0.008268
2011-01-07,0.021523,0.004684,-0.014168,0.004068,0.005456,-0.002018,0.013793,0.014388,0.006632,-0.000757,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.063791


In [10]:
prevday_returns = comb_df[new_col]
today_returns = comb_df[stocks_today]

In [11]:
print('{} {}'.format(len(new_col), len(stocks_today)))

812 713


### Define Train and Test period

In [12]:
train_start = datetime.strptime(train_start_txt, '%Y-%m-%d')
train_end = datetime.strptime(train_end_txt, '%Y-%m-%d')
train_end = train_end - relativedelta(months=3)
test_start = train_end + relativedelta(days=1)
test_end = train_end + relativedelta(months=3)
print('{} {} {} {}'.format(train_start, train_end, test_start, test_end))

2011-01-01 00:00:00 2011-09-30 00:00:00 2011-10-01 00:00:00 2011-12-30 00:00:00


### Regression

In [21]:
def get_linear_regression(pair, prevday_returns, today_returns):
    catstocks = pair['prev']
    column = pair['today']
    X = prevday_returns[catstocks].copy()
    X = sm.add_constant(X)
    y = today_returns[column]
    model = sm.OLS(y, X).fit()
    data = {
        'Prev' : catstocks,
        'Today' : column,
        'Params' : model.params.iloc[1],
        'TValues' : abs(model.tvalues.iloc[1]),
        'RSquared' : model.rsquared,
        'model' : model,
        'y_train' : y,
        'y_train_pred' : model.predict(X)
    }
    # print(data)
    return data

In [22]:
def generate_pair_list(new_col, today_returns, best_prev_sofar_df):
    catstocks = new_col
    pair_list = []
    for column in today_returns.columns:
        if best_prev_sofar_df is not None:
            best_prev_str = best_prev_sofar_df.loc[best_prev_sofar_df['Today']==column, 'Prev'].values[0]
            best_prev_list = best_prev_str.replace('\'','').strip('][').split(', ') # convert list representation to list
        else:
            best_prev_list = list()
        for catstock in catstocks:
            if (column not in catstock) and (catstock not in best_prev_list):
                data = {
                    'prev' : best_prev_list + [catstock],
                    'today' : column
                }
                pair_list.append(data)
    return pair_list
#

In [23]:
def feature_score(train_mse, test_mse, train_r2, test_r2, data):
    if data['TValues'] != data['TValues']:
        # TValues is null
        score = -1
    else:
        var = train_r2-test_r2
        if var < 0:
            var = 0
        score = train_r2 / (var+0.1)
    return score

### Train Test split wrapper

In [28]:
def linear_regression_score(pair, prevday_returns, today_returns, train_start, train_end, test_start, test_end):
    train_data = get_linear_regression(pair, prevday_returns[train_start:train_end],\
                                       today_returns[train_start:train_end])
    y_train = train_data['y_train']
    y_train_pred = train_data['y_train_pred']
    #
    X_test = prevday_returns.loc[test_start:test_end, pair['prev']].copy()
    X_test = sm.add_constant(X_test)
    y_test = today_returns.loc[test_start:test_end, pair['today']]
    y_test_pred = train_data['model'].predict(X_test)
    #
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    #
    data = {
        'Prev' : train_data['Prev'],
        'Today' : train_data['Today'],
        'Params' : train_data['Params'],
        'TValues' : train_data['TValues'],
        'RSquared' : train_data['RSquared'],
        'train_mse' : train_mse,
        'test_mse' : test_mse,
        'train_r2' : train_r2,
        'test_r2' : test_r2,
        'score' : feature_score(train_mse, test_mse, train_r2, test_r2, train_data)
    }
    # print(data)
    return data

### Generate pair list with 1 indp variable

In [ ]:
pair_list = generate_pair_list(new_col, today_returns, None)
pair_list[:5]

	Prev	Today	Params	TValues	RSquared
348463	['MFC.prev']	0945.HK	0.690853	18.223915	0.579488

### Experiment - Train Test split wrapper

In [ ]:
pair = [x for x in pair_list if (x['prev']==['MFC.prev']) and (x['today']=='0945.HK')][0]
pair

In [ ]:
train_data = get_linear_regression(pair, prevday_returns[train_start:train_end],\
                                   today_returns[train_start:train_end])
y_train = train_data['y_train']
y_train_pred = train_data['y_train_pred']
#
X_test = prevday_returns.loc[test_start:test_end, pair['prev']].copy()
X_test = sm.add_constant(X_test)
y_test = today_returns.loc[test_start:test_end, pair['today']]
y_test_pred = train_data['model'].predict(X_test)
#
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

In [ ]:
print('{} {}'.format(train_mse, test_mse))
print('{} {}'.format(train_r2, test_r2))
print('{}'.format(train_data['RSquared']))

### NON-Parallel processing

In [ ]:
%%time
results = [linear_regression_score(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list]
result_df = pd.DataFrame(results)
result_df

### Parrallel processing

In [ ]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(linear_regression_score)(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list)
results = executor(tasks)
result_df = pd.DataFrame(results)
result_df.to_csv('../data/us_hk/us_hk_traintest_1indp_regression_pair.csv')
result_df

### Read Regression Result

In [13]:
result1_df = pd.read_csv('../data/us_hk/us_hk_traintest_1indp_regression_pair.csv', index_col=0)

In [14]:
result1_df.head(5)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
0,['0002.HK.prev'],0001.HK,0.180499,1.408454,0.010961,0.000282,0.000700,0.010961,-0.056262,0.065547
1,['0003.HK.prev'],0001.HK,0.022527,0.211310,0.000249,0.000285,0.000675,0.000249,-0.019294,0.002086
2,['0004.HK.prev'],0001.HK,0.115927,1.701255,0.015912,0.000281,0.000705,0.015912,-0.064284,0.088303
3,['0005.HK.prev'],0001.HK,0.033759,0.436473,0.001063,0.000285,0.000677,0.001063,-0.022411,0.008610
4,['0006.HK.prev'],0001.HK,0.161578,1.597496,0.014057,0.000281,0.000681,0.014057,-0.028638,0.098508


In [15]:
sorted1_df = result1_df.sort_values(by='score', ascending=False)
sorted1_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351970,['^GSPC.prev'],0945.HK,1.058949,11.258758,0.414573,0.000313,0.000486,0.414573,0.447683,4.145728
351969,['^DJI.prev'],0945.HK,1.152500,10.930676,0.400294,0.000320,0.000519,0.400294,0.409329,4.002942
351960,['TWM.prev'],0945.HK,-0.383561,10.552394,0.383509,0.000329,0.000467,0.383509,0.468735,3.835091
351946,['SKF.prev'],0945.HK,-0.393532,10.422116,0.377652,0.000332,0.000464,0.377652,0.471906,3.776522
351890,['CAT.prev'],0945.HK,0.625189,10.386455,0.376042,0.000333,0.000528,0.376042,0.399955,3.760423
522279,['^DJI.prev'],2378.HK,1.230897,10.291903,0.371761,0.000412,0.000501,0.371761,0.413397,3.717607
351971,['^IXIC.prev'],0945.HK,0.909703,10.258610,0.370248,0.000336,0.000505,0.370248,0.425233,3.702485
351897,['DE.prev'],0945.HK,0.675574,10.162345,0.365863,0.000339,0.000518,0.365863,0.410718,3.658627
522280,['^GSPC.prev'],2378.HK,1.101681,10.150692,0.365330,0.000416,0.000492,0.365330,0.424316,3.653305
351941,['OMC.prev'],0945.HK,0.740291,9.510454,0.335681,0.000355,0.000453,0.335681,0.484475,3.356807


In [16]:
sorted1a_df = result1_df.sort_values(by='RSquared', ascending=False)
sorted1a_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351929,['MFC.prev'],0945.HK,0.711463,16.962874,0.616488,0.000205,0.000429,0.616488,0.512342,3.019840
351970,['^GSPC.prev'],0945.HK,1.058949,11.258758,0.414573,0.000313,0.000486,0.414573,0.447683,4.145728
351969,['^DJI.prev'],0945.HK,1.152500,10.930676,0.400294,0.000320,0.000519,0.400294,0.409329,4.002942
351960,['TWM.prev'],0945.HK,-0.383561,10.552394,0.383509,0.000329,0.000467,0.383509,0.468735,3.835091
351946,['SKF.prev'],0945.HK,-0.393532,10.422116,0.377652,0.000332,0.000464,0.377652,0.471906,3.776522
351890,['CAT.prev'],0945.HK,0.625189,10.386455,0.376042,0.000333,0.000528,0.376042,0.399955,3.760423
369211,['0373.HK.prev'],1003.HK,1.370881,10.370127,0.375304,0.007108,0.006035,0.375304,-0.346473,0.456698
522279,['^DJI.prev'],2378.HK,1.230897,10.291903,0.371761,0.000412,0.000501,0.371761,0.413397,3.717607
351971,['^IXIC.prev'],0945.HK,0.909703,10.258610,0.370248,0.000336,0.000505,0.370248,0.425233,3.702485
351897,['DE.prev'],0945.HK,0.675574,10.162345,0.365863,0.000339,0.000518,0.365863,0.410718,3.658627


# 2nd Independent variable

### Find the best result with 1st independent variable

In [17]:
best_prev_sofar_df = sorted1_df.loc[sorted1_df.groupby('Today')['score'].idxmax()]

In [18]:
best_prev_sofar_df[best_prev_sofar_df['Today']=='0945.HK']

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351970,['^GSPC.prev'],0945.HK,1.058949,11.258758,0.414573,0.000313,0.000486,0.414573,0.447683,4.145728


### Generate pair list

In [19]:
today_returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 243 entries, 2011-01-05 to 2011-12-30
Columns: 713 entries, 0001.HK to 6823.HK
dtypes: float64(713)
memory usage: 1.3 MB


In [24]:
pair_list = generate_pair_list(new_col, today_returns, best_prev_sofar_df)
pair_list[:5]

[{'prev': ['^VIX.prev', '0002.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0003.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0004.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0005.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0006.HK.prev'], 'today': '0001.HK'}]

### Double check train and test period

In [25]:
print('{} {}'.format(train_start, train_end))
print('{} {}'.format(test_start, test_end))

2011-01-01 00:00:00 2011-09-30 00:00:00
2011-10-01 00:00:00 2011-12-30 00:00:00


### NON-Parallel processing

In [ ]:
%%time
results = [linear_regression_score(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list]
result2_df = pd.DataFrame(results)
result2_df

### Parrallel processing

In [29]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(linear_regression_score)(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list)
results = executor(tasks)
result_2df = pd.DataFrame(results)
result_2df.to_csv('../data/us_hk/us_hk_traintest_2indp_regression_pair.csv')
result_2df

cpu count = 16
CPU times: user 5min 3s, sys: 13.4 s, total: 5min 16s
Wall time: 56min 54s


,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
0,"[^VIX.prev, 0002.HK.prev]",0001.HK,-0.086257,6.541266,0.202634,0.000227,0.000511,0.202634,0.228870,2.026341
1,"[^VIX.prev, 0003.HK.prev]",0001.HK,-0.087380,6.616402,0.197591,0.000229,0.000495,0.197591,0.252839,1.975913
2,"[^VIX.prev, 0004.HK.prev]",0001.HK,-0.085441,6.397064,0.199864,0.000228,0.000507,0.199864,0.233909,1.998641
3,"[^VIX.prev, 0005.HK.prev]",0001.HK,-0.088467,6.617681,0.198306,0.000229,0.000486,0.198306,0.265596,1.983057
4,"[^VIX.prev, 0006.HK.prev]",0001.HK,-0.085636,6.442574,0.200490,0.000228,0.000497,0.200490,0.249677,2.004896
...,...,...,...,...,...,...,...,...,...,...
577525,"[MYL.prev, ^DJI.prev]",6823.HK,0.360595,2.985511,0.098111,0.000380,0.102229,0.098111,-0.004307,0.484695
577526,"[MYL.prev, ^GSPC.prev]",6823.HK,0.349058,2.800781,0.097310,0.000380,0.102228,0.097310,-0.004294,0.482679
577527,"[MYL.prev, ^IXIC.prev]",6823.HK,0.369161,3.039692,0.098660,0.000379,0.102266,0.098660,-0.004673,0.485214
577528,"[MYL.prev, ^TNX.prev]",6823.HK,0.306131,3.864312,0.096310,0.000380,0.102230,0.096310,-0.004317,0.480044


### Read Regression Result

In [30]:
result2_df = pd.read_csv('../data/us_hk/us_hk_traintest_2indp_regression_pair.csv', index_col=0)

In [31]:
result2_df.head(5)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
0,"['^VIX.prev', '0002.HK.prev']",0001.HK,-0.086257,6.541266,0.202634,0.000227,0.000511,0.202634,0.228870,2.026341
1,"['^VIX.prev', '0003.HK.prev']",0001.HK,-0.087380,6.616402,0.197591,0.000229,0.000495,0.197591,0.252839,1.975913
2,"['^VIX.prev', '0004.HK.prev']",0001.HK,-0.085441,6.397064,0.199864,0.000228,0.000507,0.199864,0.233909,1.998641
3,"['^VIX.prev', '0005.HK.prev']",0001.HK,-0.088467,6.617681,0.198306,0.000229,0.000486,0.198306,0.265596,1.983057
4,"['^VIX.prev', '0006.HK.prev']",0001.HK,-0.085636,6.442574,0.200490,0.000228,0.000497,0.200490,0.249677,2.004896


In [32]:
sorted2_df = result2_df.sort_values(by='score', ascending=False)
sorted2_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351330,"['^GSPC.prev', '2000.HK.prev']",0945.HK,1.091866,11.786864,0.443121,0.000297,0.000405,0.443121,0.539471,4.431206
350851,"['^GSPC.prev', '0218.HK.prev']",0945.HK,1.080743,11.687433,0.441012,0.000299,0.000490,0.441012,0.442357,4.410118
351211,"['^GSPC.prev', '1083.HK.prev']",0945.HK,1.047780,11.322528,0.437587,0.000300,0.000485,0.437587,0.448539,4.375874
351353,"['^GSPC.prev', '2314.HK.prev']",0945.HK,1.091844,11.707821,0.437496,0.000300,0.000476,0.437496,0.458445,4.374956
351350,"['^GSPC.prev', '2302.HK.prev']",0945.HK,1.060334,11.459217,0.436580,0.000301,0.000473,0.436580,0.461410,4.365803
350964,"['^GSPC.prev', '0444.HK.prev']",0945.HK,1.049307,11.321732,0.435658,0.000301,0.000471,0.435658,0.464075,4.356577
350850,"['^GSPC.prev', '0217.HK.prev']",0945.HK,1.062509,11.465108,0.434959,0.000302,0.000468,0.434959,0.467889,4.349595
351241,"['^GSPC.prev', '1157.HK.prev']",0945.HK,1.076426,11.584375,0.434942,0.000302,0.000485,0.434942,0.448228,4.349421
350955,"['^GSPC.prev', '0413.HK.prev']",0945.HK,1.061522,11.437736,0.433249,0.000303,0.000492,0.433249,0.440101,4.332495
350977,"['^GSPC.prev', '0489.HK.prev']",0945.HK,1.072096,11.529007,0.432920,0.000303,0.000443,0.432920,0.495547,4.329198


In [33]:
sorted1_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351970,['^GSPC.prev'],0945.HK,1.058949,11.258758,0.414573,0.000313,0.000486,0.414573,0.447683,4.145728
351969,['^DJI.prev'],0945.HK,1.152500,10.930676,0.400294,0.000320,0.000519,0.400294,0.409329,4.002942
351960,['TWM.prev'],0945.HK,-0.383561,10.552394,0.383509,0.000329,0.000467,0.383509,0.468735,3.835091
351946,['SKF.prev'],0945.HK,-0.393532,10.422116,0.377652,0.000332,0.000464,0.377652,0.471906,3.776522
351890,['CAT.prev'],0945.HK,0.625189,10.386455,0.376042,0.000333,0.000528,0.376042,0.399955,3.760423
522279,['^DJI.prev'],2378.HK,1.230897,10.291903,0.371761,0.000412,0.000501,0.371761,0.413397,3.717607
351971,['^IXIC.prev'],0945.HK,0.909703,10.258610,0.370248,0.000336,0.000505,0.370248,0.425233,3.702485
351897,['DE.prev'],0945.HK,0.675574,10.162345,0.365863,0.000339,0.000518,0.365863,0.410718,3.658627
522280,['^GSPC.prev'],2378.HK,1.101681,10.150692,0.365330,0.000416,0.000492,0.365330,0.424316,3.653305
351941,['OMC.prev'],0945.HK,0.740291,9.510454,0.335681,0.000355,0.000453,0.335681,0.484475,3.356807


# 3rd Independent variable

### Find the best result with 2nd independent variable

In [35]:
best_prev_sofar_df = sorted2_df.loc[sorted2_df.groupby('Today')['score'].idxmax()]

In [36]:
best_prev_sofar_df[best_prev_sofar_df['Today']=='0945.HK']

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351330,"['^GSPC.prev', '2000.HK.prev']",0945.HK,1.091866,11.786864,0.443121,0.000297,0.000405,0.443121,0.539471,4.431206


### Generate pair list

In [37]:
today_returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 243 entries, 2011-01-05 to 2011-12-30
Columns: 713 entries, 0001.HK to 6823.HK
dtypes: float64(713)
memory usage: 1.3 MB


In [38]:
pair_list = generate_pair_list(new_col, today_returns, best_prev_sofar_df)
pair_list[:5]

[{'prev': ['^VIX.prev', '0050.HK.prev', '0002.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0050.HK.prev', '0003.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0050.HK.prev', '0004.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0050.HK.prev', '0005.HK.prev'], 'today': '0001.HK'},
 {'prev': ['^VIX.prev', '0050.HK.prev', '0006.HK.prev'], 'today': '0001.HK'}]

### Double check train and test period

In [39]:
print('{} {}'.format(train_start, train_end))
print('{} {}'.format(test_start, test_end))

2011-01-01 00:00:00 2011-09-30 00:00:00
2011-10-01 00:00:00 2011-12-30 00:00:00


### NON-Parallel processing

In [ ]:
%%time
results = [linear_regression_score(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list]
result3_df = pd.DataFrame(results)
result3_df

### Parrallel processing

In [40]:
%%time
cpu = cpu_count()
print('cpu count = {}'.format(cpu))
executor = Parallel(n_jobs=cpu, backend='multiprocessing')
tasks = (delayed(linear_regression_score)(pair, prevday_returns, today_returns,train_start, train_end, test_start, test_end) for pair in pair_list)
results = executor(tasks)
result_3df = pd.DataFrame(results)
result_3df.to_csv('../data/us_hk/us_hk_traintest_3indp_regression_pair.csv')
result_3df

cpu count = 16
CPU times: user 5min 4s, sys: 14.2 s, total: 5min 18s
Wall time: 1h 42s


,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
0,"[^VIX.prev, 0050.HK.prev, 0002.HK.prev]",0001.HK,-0.093677,7.202451,0.250741,0.000214,0.000533,0.250741,0.195289,1.612975
1,"[^VIX.prev, 0050.HK.prev, 0003.HK.prev]",0001.HK,-0.095225,7.226468,0.236690,0.000218,0.000504,0.236690,0.239505,2.366904
2,"[^VIX.prev, 0050.HK.prev, 0004.HK.prev]",0001.HK,-0.091069,6.945609,0.246212,0.000215,0.000519,0.246212,0.215892,1.889285
3,"[^VIX.prev, 0050.HK.prev, 0005.HK.prev]",0001.HK,-0.092874,7.023441,0.231371,0.000219,0.000498,0.231371,0.248225,2.313715
4,"[^VIX.prev, 0050.HK.prev, 0006.HK.prev]",0001.HK,-0.091807,6.973427,0.239141,0.000217,0.000506,0.239141,0.236787,2.336408
...,...,...,...,...,...,...,...,...,...,...
576812,"[MYL.prev, 1223.HK.prev, ^DJI.prev]",6823.HK,0.424714,3.534825,0.141066,0.000362,0.102327,0.141066,-0.005264,0.572670
576813,"[MYL.prev, 1223.HK.prev, ^GSPC.prev]",6823.HK,0.410736,3.316116,0.139038,0.000362,0.102323,0.139038,-0.005226,0.569212
576814,"[MYL.prev, 1223.HK.prev, ^IXIC.prev]",6823.HK,0.414339,3.453535,0.139921,0.000362,0.102369,0.139921,-0.005685,0.569696
576815,"[MYL.prev, 1223.HK.prev, ^TNX.prev]",6823.HK,0.311517,4.004525,0.134045,0.000365,0.102298,0.134045,-0.004984,0.560791


### Read Regression Result

In [41]:
result3_df = pd.read_csv('../data/us_hk/us_hk_traintest_3indp_regression_pair.csv', index_col=0)

In [42]:
result3_df.head(5)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
0,"['^VIX.prev', '0050.HK.prev', '0002.HK.prev']",0001.HK,-0.093677,7.202451,0.250741,0.000214,0.000533,0.250741,0.195289,1.612975
1,"['^VIX.prev', '0050.HK.prev', '0003.HK.prev']",0001.HK,-0.095225,7.226468,0.236690,0.000218,0.000504,0.236690,0.239505,2.366904
2,"['^VIX.prev', '0050.HK.prev', '0004.HK.prev']",0001.HK,-0.091069,6.945609,0.246212,0.000215,0.000519,0.246212,0.215892,1.889285
3,"['^VIX.prev', '0050.HK.prev', '0005.HK.prev']",0001.HK,-0.092874,7.023441,0.231371,0.000219,0.000498,0.231371,0.248225,2.313715
4,"['^VIX.prev', '0050.HK.prev', '0006.HK.prev']",0001.HK,-0.091807,6.973427,0.239141,0.000217,0.000506,0.239141,0.236787,2.336408


In [43]:
sorted3_df = result3_df.sort_values(by='score', ascending=False)
sorted3_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
350575,"['^GSPC.prev', '2000.HK.prev', '0560.HK.prev']",0945.HK,1.044734,11.410420,0.473203,0.000281,0.000437,0.473203,0.503464,4.732032
350404,"['^GSPC.prev', '2000.HK.prev', '0195.HK.prev']",0945.HK,1.051963,11.504704,0.471337,0.000282,0.000418,0.471337,0.524977,4.713373
350724,"['^GSPC.prev', '2000.HK.prev', '0931.HK.prev']",0945.HK,1.102941,12.172810,0.471066,0.000283,0.000425,0.471066,0.516588,4.710664
350551,"['^GSPC.prev', '2000.HK.prev', '0508.HK.prev']",0945.HK,1.068921,11.763917,0.470866,0.000283,0.000427,0.470866,0.514058,4.708661
350522,"['^GSPC.prev', '2000.HK.prev', '0413.HK.prev']",0945.HK,1.100370,12.145030,0.470777,0.000283,0.000404,0.470777,0.540689,4.707766
350466,"['^GSPC.prev', '2000.HK.prev', '0305.HK.prev']",0945.HK,1.052504,11.488551,0.469670,0.000283,0.000406,0.469670,0.538083,4.696703
350848,"['^GSPC.prev', '2000.HK.prev', '1368.HK.prev']",0945.HK,1.064893,11.685707,0.469456,0.000283,0.000439,0.469456,0.500687,4.694556
350941,"['^GSPC.prev', '2000.HK.prev', '2383.HK.prev']",0945.HK,1.065447,11.685746,0.468817,0.000284,0.000412,0.468817,0.531184,4.688168
350852,"['^GSPC.prev', '2000.HK.prev', '1386.HK.prev']",0945.HK,1.131427,12.306962,0.467210,0.000285,0.000399,0.467210,0.545894,4.672097
350981,"['^GSPC.prev', '2000.HK.prev', '3378.HK.prev']",0945.HK,1.100984,12.106057,0.466940,0.000285,0.000457,0.466940,0.480080,4.669396


In [44]:
sorted2_df.head(10)

,Prev,Today,Params,TValues,RSquared,train_mse,test_mse,train_r2,test_r2,score
351330,"['^GSPC.prev', '2000.HK.prev']",0945.HK,1.091866,11.786864,0.443121,0.000297,0.000405,0.443121,0.539471,4.431206
350851,"['^GSPC.prev', '0218.HK.prev']",0945.HK,1.080743,11.687433,0.441012,0.000299,0.000490,0.441012,0.442357,4.410118
351211,"['^GSPC.prev', '1083.HK.prev']",0945.HK,1.047780,11.322528,0.437587,0.000300,0.000485,0.437587,0.448539,4.375874
351353,"['^GSPC.prev', '2314.HK.prev']",0945.HK,1.091844,11.707821,0.437496,0.000300,0.000476,0.437496,0.458445,4.374956
351350,"['^GSPC.prev', '2302.HK.prev']",0945.HK,1.060334,11.459217,0.436580,0.000301,0.000473,0.436580,0.461410,4.365803
350964,"['^GSPC.prev', '0444.HK.prev']",0945.HK,1.049307,11.321732,0.435658,0.000301,0.000471,0.435658,0.464075,4.356577
350850,"['^GSPC.prev', '0217.HK.prev']",0945.HK,1.062509,11.465108,0.434959,0.000302,0.000468,0.434959,0.467889,4.349595
351241,"['^GSPC.prev', '1157.HK.prev']",0945.HK,1.076426,11.584375,0.434942,0.000302,0.000485,0.434942,0.448228,4.349421
350955,"['^GSPC.prev', '0413.HK.prev']",0945.HK,1.061522,11.437736,0.433249,0.000303,0.000492,0.433249,0.440101,4.332495
350977,"['^GSPC.prev', '0489.HK.prev']",0945.HK,1.072096,11.529007,0.432920,0.000303,0.000443,0.432920,0.495547,4.329198


### Upload to AWS S3

In [45]:
s3_bucket = 'slack-trading'
local_path = '../data/us_hk'
tools.upload_data_to_s3(s3_bucket, local_path)